In [4]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
from tqdm import tqdm
import os
import numpy as np

# --- Constants ---
CASE_URL = "https://api.vitaldb.net/cases"
TRACK_URL = "https://api.vitaldb.net/trks"
DATA_URL = "https://api.vitaldb.net/{}"
OUTPUT_DIR = "case_hr_monitor"

PARAMETERS = {
    'Solar8000/HR': 'Heart Rate',
    'Solar8000/ART_MBP': 'Arterial BP',
    'Solar8000/PLETH_SPO2': 'Oxygen Saturation',
    'Solar8000/ETCO2': 'End-Tidal CO2',
    'Solar8000/RR': 'Respiratory Rate'
}

# --- Create output directory ---
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Fetch first 10 cases as CSV ---
cases_df = pd.read_csv(CASE_URL)
first_10_caseids = cases_df['caseid'].head(10).tolist()

# --- Fetch track list ---
tracks_df = pd.read_csv(TRACK_URL)

# --- Filter tracks ---
filtered_tracks = tracks_df[
    (tracks_df['caseid'].isin(first_10_caseids)) &
    (tracks_df['tname'].isin(PARAMETERS.keys()))
]

# --- Group by caseid ---
case_grouped_tracks = filtered_tracks.groupby('caseid')

# --- Plotting ---
for caseid, case_tracks in tqdm(case_grouped_tracks, desc="Rendering 15-Min Trend Plots"):
    fig, (ax_hr, ax_params) = plt.subplots(2, 1, figsize=(12, 6), sharex=True, gridspec_kw={'height_ratios': [2, 1]})
    fig.suptitle(f"Case {caseid}: Trends Every 15 Minutes")

    for _, row in case_tracks.iterrows():
        tid = row['tid']
        label = PARAMETERS[row['tname']]
        try:
            df = pd.read_csv(DATA_URL.format(tid), header=None, names=['time', 'value'])
            df['time'] = pd.to_numeric(df['time'], errors='coerce')
            df['value'] = pd.to_numeric(df['value'], errors='coerce')
            df = df.dropna(subset=['time', 'value'])
            df = df.sort_values('time')

            # Bin data into 15-minute intervals (900s)
            df['bin'] = (df['time'] // 900).astype(int)
            df_avg = df.groupby('bin')['value'].mean().reset_index()
            df_avg['time'] = df_avg['bin'] * 900

            if label == 'Heart Rate':
                ax_hr.plot(df_avg['time'], df_avg['value'], label=label, color='red')
                ax_hr.set_ylabel("BPM")
                ax_hr.set_title("Heart Rate")
            else:
                ax_params.plot(df_avg['time'], df_avg['value'], label=label)
        except Exception as e:
            print(f"Error loading TID {tid}: {e}")

    ax_hr.legend(loc='upper right')
    ax_params.legend(loc='upper right')
    ax_params.set_ylabel("Value")
    ax_params.set_xlabel("Time (s)")
    ax_params.set_title("Other Parameters")

    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig.savefig(os.path.join(OUTPUT_DIR, f"case_{caseid}_monitor.png"))
    plt.close(fig)

Rendering 15-Min Trend Plots: 100%|██████████| 10/10 [00:02<00:00,  3.73it/s]


In [3]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
from tqdm import tqdm
import os
import numpy as np

# --- Constants ---
CASE_URL = "https://api.vitaldb.net/cases"
TRACK_URL = "https://api.vitaldb.net/trks"
DATA_URL = "https://api.vitaldb.net/{}"
OUTPUT_DIR = "case_15min_avg_trends"

PARAMETERS = {
    'Solar8000/HR': 'Heart Rate',
    'Solar8000/ART_MBP': 'Arterial BP',
    'Solar8000/PLETH_SPO2': 'Oxygen Saturation',
    'Solar8000/ETCO2': 'End-Tidal CO2',
    'Solar8000/RR': 'Respiratory Rate'
}

COLOR_MAP = {
    'Heart Rate': 'red',
    'Arterial BP': 'green',
    'Oxygen Saturation': 'blue',
    'End-Tidal CO2': 'purple',
    'Respiratory Rate': 'yellow'
}

# --- Create output directory ---
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Fetch first 10 cases as CSV ---
cases_df = pd.read_csv(CASE_URL)
first_10_caseids = cases_df['caseid'].head(50).tolist()

# --- Fetch track list ---
tracks_df = pd.read_csv(TRACK_URL)

# --- Filter tracks ---
filtered_tracks = tracks_df[
    (tracks_df['caseid'].isin(first_10_caseids)) &
    (tracks_df['tname'].isin(PARAMETERS.keys()))
]

# --- Group by caseid ---
case_grouped_tracks = filtered_tracks.groupby('caseid')

# --- Plotting ---
for caseid, case_tracks in tqdm(case_grouped_tracks, desc="Rendering 15-Min Trend Plots"):
    fig, ax = plt.subplots(figsize=(12, 5))
    fig.suptitle(f"Case {caseid}: Vital Trends Averaged Every 15 Minutes")

    for _, row in case_tracks.iterrows():
        tid = row['tid']
        label = PARAMETERS[row['tname']]
        try:
            df = pd.read_csv(DATA_URL.format(tid), header=None, names=['time', 'value'])
            df['time'] = pd.to_numeric(df['time'], errors='coerce')
            df['value'] = pd.to_numeric(df['value'], errors='coerce')
            df = df.dropna(subset=['time', 'value'])
            df = df.sort_values('time')

            df['bin'] = (df['time'] // 900).astype(int)
            df_avg = df.groupby('bin')['value'].mean().reset_index()
            df_avg['time'] = df_avg['bin'] * 900

            ax.plot(df_avg['time'], df_avg['value'], label=label, color=COLOR_MAP[label])
        except Exception as e:
            print(f"Error loading TID {tid}: {e}")

    ax.legend(loc='upper right')
    ax.set_ylabel("Value")
    ax.set_xlabel("Time (s)")
    ax.set_title("Vitals (Averaged Every 15 Minutes)")

    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig.savefig(os.path.join(OUTPUT_DIR, f"case_{caseid}_trends.png"))
    plt.close(fig)

Rendering 15-Min Trend Plots: 100%|██████████| 50/50 [00:37<00:00,  1.32it/s]


In [5]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
from tqdm import tqdm
import os
import numpy as np
import math

# --- Constants ---
CASE_URL = "https://api.vitaldb.net/cases"
TRACK_URL = "https://api.vitaldb.net/trks"
DATA_URL = "https://api.vitaldb.net/{}"
OUTPUT_DIR = "case_15min_avg_trends_all"

# --- Create output directory ---
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Fetch first 10 cases as CSV ---
cases_df = pd.read_csv(CASE_URL)
first_10_caseids = cases_df['caseid'].head(10).tolist()

# --- Fetch track list ---
tracks_df = pd.read_csv(TRACK_URL)

# --- Filter tracks for first 10 caseids ---
filtered_tracks = tracks_df[tracks_df['caseid'].isin(first_10_caseids)]

# --- Group by caseid ---
case_grouped_tracks = filtered_tracks.groupby('caseid')

# --- Plotting ---
for caseid, case_tracks in tqdm(case_grouped_tracks, desc="Rendering All-Param 15-Min Trend Subplots"):
    num_tracks = len(case_tracks)
    cols = 3
    rows = math.ceil(num_tracks / cols)
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 3))
    fig.suptitle(f"Case {caseid}: All Vitals Averaged Every 15 Minutes", fontsize=16)
    axes = axes.flatten() if num_tracks > 1 else [axes]

    for i, (_, row) in enumerate(case_tracks.iterrows()):
        tid = row['tid']
        label = row['tname']
        ax = axes[i]
        try:
            df = pd.read_csv(DATA_URL.format(tid), header=None, names=['time', 'value'])
            df['time'] = pd.to_numeric(df['time'], errors='coerce')
            df['value'] = pd.to_numeric(df['value'], errors='coerce')
            df = df.dropna(subset=['time', 'value'])
            df = df.sort_values('time')

            df['bin'] = (df['time'] // 900).astype(int)
            df_avg = df.groupby('bin')['value'].mean().reset_index()
            df_avg['time'] = df_avg['bin'] * 900

            ax.plot(df_avg['time'], df_avg['value'])
            ax.set_title(label, fontsize=10)
            ax.set_xlabel("Time (s)", fontsize=8)
            ax.set_ylabel("Value", fontsize=8)
            ax.tick_params(axis='both', labelsize=8)
        except Exception as e:
            ax.set_title(f"{label} (Error)", fontsize=10)
            print(f"Error loading TID {tid}: {e}")

    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig.savefig(os.path.join(OUTPUT_DIR, f"case_{caseid}_all_trends.png"))
    plt.close(fig)



Rendering All-Param 15-Min Trend Subplots:   0%|          | 0/10 [00:00<?, ?it/s]C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_32872\413966400.py:46: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_URL.format(tid), header=None, names=['time', 'value'])
C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_32872\413966400.py:46: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_URL.format(tid), header=None, names=['time', 'value'])
C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_32872\413966400.py:46: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_URL.format(tid), header=None, names=['time', 'value'])
C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_32872\413966400.py:46: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=F

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
from tqdm import tqdm
import os
import numpy as np
import math

# --- Constants ---
CASE_URL = "https://api.vitaldb.net/cases"
TRACK_URL = "https://api.vitaldb.net/trks"
DATA_URL = "https://api.vitaldb.net/{}"
OUTPUT_DIR = "case_15min_avg_trends_all"
CORR_OUTPUT = "hr_correlations.csv"

# --- Create output directory ---
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Fetch first 100 cases as CSV ---
cases_df = pd.read_csv(CASE_URL)
first_100_caseids = cases_df['caseid'].head(100).tolist()

# --- Fetch track list ---
tracks_df = pd.read_csv(TRACK_URL)

# --- Filter tracks for first 100 caseids ---
filtered_tracks = tracks_df[tracks_df['caseid'].isin(first_100_caseids)]

# --- Group by caseid ---
case_grouped_tracks = filtered_tracks.groupby('caseid')

# --- Store HR correlation values across all cases ---
hr_corrs = []

# --- Plotting & Correlation ---
for caseid, case_tracks in tqdm(case_grouped_tracks, desc="Processing Cases"):
    num_tracks = len(case_tracks)
    cols = 3
    rows = math.ceil(num_tracks / cols)
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 3))
    fig.suptitle(f"Case {caseid}: All Vitals Averaged Every 15 Minutes", fontsize=16)
    axes = axes.flatten() if num_tracks > 1 else [axes]

    time_series = {}

    for i, (_, row) in enumerate(case_tracks.iterrows()):
        tid = row['tid']
        label = row['tname']
        ax = axes[i]
        try:
            df = pd.read_csv(DATA_URL.format(tid), header=None, names=['time', 'value'])
            df['time'] = pd.to_numeric(df['time'], errors='coerce')
            df['value'] = pd.to_numeric(df['value'], errors='coerce')
            df = df.dropna(subset=['time', 'value'])
            df = df.sort_values('time')

            df['bin'] = (df['time'] // 900).astype(int)
            df_avg = df.groupby('bin')['value'].mean().reset_index()
            df_avg['time'] = df_avg['bin'] * 900
            time_series[label] = df_avg.set_index('time')['value']

            ax.plot(df_avg['time'], df_avg['value'])
            ax.set_title(label, fontsize=10)
            ax.set_xlabel("Time (s)", fontsize=8)
            ax.set_ylabel("Value", fontsize=8)
            ax.tick_params(axis='both', labelsize=8)
        except Exception as e:
            ax.set_title(f"{label} (Error)", fontsize=10)
            print(f"Error loading TID {tid}: {e}")

    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig.savefig(os.path.join(OUTPUT_DIR, f"case_{caseid}_all_trends.png"))
    plt.close(fig)

    # --- Correlation with Heart Rate ---
    if 'Solar8000/HR' in case_tracks['tname'].values:
        df_combined = pd.DataFrame(time_series).dropna(axis=1, how='all').dropna()
        if 'Solar8000/HR' in df_combined.columns:
            corr_series = df_combined.corr(method='pearson')['Solar8000/HR'].drop('Solar8000/HR', errors='ignore')
            for feature, corr in corr_series.items():
                hr_corrs.append({'caseid': caseid, 'feature': feature, 'correlation': corr})

# --- Save correlation results to CSV ---
pd.DataFrame(hr_corrs).to_csv(CORR_OUTPUT, index=False)


Processing Cases:   0%|          | 0/100 [00:00<?, ?it/s]C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_32872\380058011.py:52: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_URL.format(tid), header=None, names=['time', 'value'])
C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_32872\380058011.py:52: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_URL.format(tid), header=None, names=['time', 'value'])
C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_32872\380058011.py:52: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_URL.format(tid), header=None, names=['time', 'value'])
C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_32872\380058011.py:52: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv

Error loading TID ac45a27c76254ad308dd8eb4c725a193bbfdb63d: Remote end closed connection without response


C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_32872\380058011.py:52: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_URL.format(tid), header=None, names=['time', 'value'])
C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_32872\380058011.py:52: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_URL.format(tid), header=None, names=['time', 'value'])
C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_32872\380058011.py:52: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_URL.format(tid), header=None, names=['time', 'value'])
C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_32872\380058011.py:52: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_URL.format(tid), header=None, names=['time', 'value